In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam

In [ ]:
def load_data_from_mat(file_path, key):
    data = loadmat(file_path)
    return data[key]


In [ ]:
# Define paths and keys for each class
file_info_dict = {
    'A': [('/content/drive/MyDrive/emdvmd/VMDF.mat', 'IMFvmd1F')],
    'B': [('/content/drive/MyDrive/emdvmd/VMDN.mat', 'IMFvmd1N')],
    'C': [('/content/drive/MyDrive/emdvmd/VMDO.mat', 'IMFvmd1O')],
    'D': [('/content/drive/MyDrive/emdvmd/VMDS.mat', 'IMFvmd1S')],
    'E': [('/content/drive/MyDrive/emdvmd/VMDZ.mat', 'IMFvmd1Z')]
}

In [ ]:
def prepare_data_for_classification(classes):
    X, y = [], []
    for label, class_id in enumerate(classes):
        file_info = file_info_dict[class_id]  # Get file paths and keys for each class
        for file_path, key in file_info:
            data = load_data_from_mat(file_path, key)
            X.append(data)
            y.append(np.array([label] * data.shape[0]))  # Assign the same label to all samples
    X = np.concatenate(X)
    y = np.concatenate(y)
    return X, y


In [ ]:
def create_ann_model(input_shape, dense_units1, dense_units2, output_units, dropout_rate):
    model = Sequential()
    model.add(Dense(units=dense_units1, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(units=dense_units2, activation='relu'))
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(units=output_units, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def compute_metrics(y_true, y_pred):
    y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average='macro')
    specificity = recall_score(y_true, y_pred, average='macro', labels=[0, 1])
    f1 = f1_score(y_true, y_pred, average='macro')
    return accuracy, sensitivity, specificity, f1

In [ ]:
def train_and_evaluate_ann_model(X_train, y_train, X_test, y_test, dense_units1, dense_units2, dropout_rate, batch_size, epochs):
    model = create_ann_model(X_train.shape[1], dense_units1, dense_units2, y_train.shape[1], dropout_rate)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=1)
    y_pred = model.predict(X_test)
    score = model.evaluate(X_test, y_test, verbose=1)
    accuracy, sensitivity, specificity, f1 = compute_metrics(y_test, y_pred)
    print(f'Test loss: {score[0]}')
    print(f'Test accuracy: {accuracy}')
    print(f'Sensitivity (Recall): {sensitivity}')
    print(f'Specificity: {specificity}')
    print(f'F1 Score: {f1}')

Classification A-B vs. C-D-E

In [ ]:
# Load and preprocess data for classes A-B vs. C-D-E
classes_ab = ['A', 'B']
classes_cde = ['C', 'D', 'E']

X_ab, y_ab = prepare_data_for_classification(classes_ab)
X_cde, y_cde = prepare_data_for_classification(classes_cde)

# Merge and create new labels
X = np.concatenate([X_ab, X_cde])
y = np.concatenate([np.zeros(len(y_ab)), np.ones(len(y_cde))])  # 0 for A-B, 1 for C-D-E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
y = y.astype(int)  # Ensure labels are integers
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the ANN model
train_and_evaluate_ann_model(X_train, y_train, X_test, y_test, dense_units1=128, dense_units2=64, dropout_rate=0.5, batch_size=32, epochs=20)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6494 - loss: 0.5924 - val_accuracy: 0.9564 - val_loss: 0.2933
Epoch 2/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9037 - loss: 0.2760 - val_accuracy: 0.9918 - val_loss: 0.0619
Epoch 3/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9630 - loss: 0.1145 - val_accuracy: 0.9948 - val_loss: 0.0263
Epoch 4/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9797 - loss: 0.0607 - val_accuracy: 0.9948 - val_loss: 0.0178
Epoch 5/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9859 - loss: 0.0420 - val_accuracy: 0.9969 - val_loss: 0.0111
Epoch 6/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9905 - loss: 0.0312 - val_accuracy: 0.9973 - val_loss: 0.0140
Epoch 7/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9908 - loss: 0.0247 - val_accuracy: 0.9976 - val_loss: 0.0079
Epoch 8/20
410/410 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9936 - loss: 0.0210 - val_accuracy: 0.

Classification A vs. E

In [ ]:
# Load and preprocess data for classes A vs. E
classes_a = ['A']
classes_e = ['E']

X_a, y_a = prepare_data_for_classification(classes_a)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_a, X_e])
y = np.concatenate([np.zeros(len(y_a)), np.ones(len(y_e))])  # 0 for A, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the ANN model
train_and_evaluate_ann_model(X_train, y_train, X_test, y_test, dense_units1=128, dense_units2=64, dropout_rate=0.5, batch_size=32, epochs=20)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


164/164 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5353 - loss: 0.8478 - val_accuracy: 0.8391 - val_loss: 0.5584
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6957 - loss: 0.5796 - val_accuracy: 0.9458 - val_loss: 0.3801
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8271 - loss: 0.3931 - val_accuracy: 0.9794 - val_loss: 0.1792
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8920 - loss: 0.2660 - val_accuracy: 0.9916 - val_loss: 0.0850
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9405 - loss: 0.1635 - val_accuracy: 0.9954 - val_loss: 0.0423
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9625 - loss: 0.1028 - val_accuracy: 0.9962 - val_loss: 0.0242
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9791 - loss: 0.0725 - val_accuracy: 0.9969 - val_loss: 0.0169
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9802 - loss: 0.0605 - val_accuracy: 0.9977 - val_

Classification B vs. E

In [ ]:
# Load and preprocess data for classes B vs. E
classes_b = ['B']
classes_e = ['E']

X_b, y_b = prepare_data_for_classification(classes_b)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_b, X_e])
y = np.concatenate([np.zeros(len(y_b)), np.ones(len(y_e))])  # 0 for B, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the ANN model
train_and_evaluate_ann_model(X_train, y_train, X_test, y_test, dense_units1=128, dense_units2=64, dropout_rate=0.5, batch_size=32, epochs=20)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.5255 - loss: 0.8772 - val_accuracy: 0.8078 - val_loss: 0.5780
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6650 - loss: 0.6002 - val_accuracy: 0.9314 - val_loss: 0.4507
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7931 - loss: 0.4489 - val_accuracy: 0.9817 - val_loss: 0.2246
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8896 - loss: 0.2757 - val_accuracy: 0.9992 - val_loss: 0.0734
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9376 - loss: 0.1716 - val_accuracy: 1.0000 - val_loss: 0.0310
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9596 - loss: 0.1147 - val_accuracy: 1.0000 - val_loss: 0.0137
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9748 - loss: 0.0780 - val_accuracy: 1.0000 - val_loss: 0.0081
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9801 - loss: 0.0607 - val_accuracy: 1.

Classification B vs. D

In [ ]:
# Load and preprocess data for classes B vs. D
classes_b = ['B']
classes_d = ['D']

X_b, y_b = prepare_data_for_classification(classes_b)
X_d, y_d = prepare_data_for_classification(classes_d)

# Merge and create new labels
X = np.concatenate([X_b, X_d])
y = np.concatenate([np.zeros(len(y_b)), np.ones(len(y_d))])  # 0 for B, 1 for D

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the ANN model
train_and_evaluate_ann_model(X_train, y_train, X_test, y_test, dense_units1=128, dense_units2=64, dropout_rate=0.5, batch_size=32, epochs=20)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5853 - loss: 0.7436 - val_accuracy: 0.9947 - val_loss: 0.2321
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9603 - loss: 0.2226 - val_accuracy: 1.0000 - val_loss: 0.0344
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9913 - loss: 0.0617 - val_accuracy: 1.0000 - val_loss: 0.0090
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9962 - loss: 0.0299 - val_accuracy: 1.0000 - val_loss: 0.0033
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9989 - loss: 0.0113 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9998 - loss: 0.0072 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 1.0000 - val_loss: 5.9307e-04
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9998 - loss: 0.0035 - val_accuracy: 1.0000 - 

Classification A vs. C

In [ ]:
# Load and preprocess data for classes A vs. C
classes_a = ['A']
classes_c = ['C']

X_a, y_a = prepare_data_for_classification(classes_a)
X_c, y_c = prepare_data_for_classification(classes_c)

# Merge and create new labels
X = np.concatenate([X_a, X_c])
y = np.concatenate([np.zeros(len(y_a)), np.ones(len(y_c))])  # 0 for A, 1 for C

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the ANN model
train_and_evaluate_ann_model(X_train, y_train, X_test, y_test, dense_units1=128, dense_units2=64, dropout_rate=0.5, batch_size=32, epochs=20)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


164/164 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.5501 - loss: 0.7931 - val_accuracy: 0.8825 - val_loss: 0.5062
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7238 - loss: 0.5421 - val_accuracy: 0.9771 - val_loss: 0.2863
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8535 - loss: 0.3496 - val_accuracy: 0.9901 - val_loss: 0.1103
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9373 - loss: 0.1837 - val_accuracy: 0.9985 - val_loss: 0.0362
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9690 - loss: 0.1055 - val_accuracy: 1.0000 - val_loss: 0.0159
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9769 - loss: 0.0704 - val_accuracy: 1.0000 - val_loss: 0.0077
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9853 - loss: 0.0498 - val_accuracy: 1.0000 - val_loss: 0.0049
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9928 - loss: 0.0308 - val_accuracy: 1.0000 - val_

Classification C vs. E

In [ ]:
# Load and preprocess data for classes C vs. E
classes_c = ['C']
classes_e = ['E']

X_c, y_c = prepare_data_for_classification(classes_c)
X_e, y_e = prepare_data_for_classification(classes_e)

# Merge and create new labels
X = np.concatenate([X_c, X_e])
y = np.concatenate([np.zeros(len(y_c)), np.ones(len(y_e))])  # 0 for C, 1 for E

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode labels
y = y.astype(int)
y = np.eye(len(np.unique(y)))[y]  # One-hot encode the labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and evaluate the ANN model
train_and_evaluate_ann_model(X_train, y_train, X_test, y_test, dense_units1=128, dense_units2=64, dropout_rate=0.5, batch_size=32, epochs=20)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


164/164 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5100 - loss: 0.8842 - val_accuracy: 0.7368 - val_loss: 0.6271
Epoch 2/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6006 - loss: 0.6525 - val_accuracy: 0.8696 - val_loss: 0.5618
Epoch 3/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6996 - loss: 0.5645 - val_accuracy: 0.9481 - val_loss: 0.4082
Epoch 4/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7969 - loss: 0.4473 - val_accuracy: 0.9817 - val_loss: 0.2305
Epoch 5/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8660 - loss: 0.3150 - val_accuracy: 0.9901 - val_loss: 0.1202
Epoch 6/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9174 - loss: 0.2201 - val_accuracy: 0.9947 - val_loss: 0.0645
Epoch 7/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9407 - loss: 0.1578 - val_accuracy: 0.9962 - val_loss: 0.0354
Epoch 8/20
164/164 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9624 - loss: 0.1115 - val_accuracy: 0.9992 - val_